In [8]:
# Import necessary libraries
import pandas as pd
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.preprocessing import StandardScaler

In [ ]:
# Load datasets
customers_data = pd.DataFrame({
    "CustomerID": ["C0001", "C0002", "C0003", "C0004"],  # Include your data here
    "CustomerName": ["Lawrence Carroll", "Elizabeth Lutz", "Michael Rivera", "Kathleen Rodriguez"],
    "Region": ["South America", "Asia", "South America", "South America"],
    "SignupDate": ["7/10/2022", "2/13/2022", "3/7/2024", "10/9/2022"]
})

transactions_file = "/mnt/data/Transactions.csv"
transactions_data = pd.read_csv(transactions_file)

In [ ]:
# Preprocessing customer and transaction data
customer_transactions = pd.merge(transactions_data, customers_data, on="CustomerID")
customer_summary = customer_transactions.groupby("CustomerID").agg(
    total_spent=("TotalValue", "sum"),
    avg_quantity=("Quantity", "mean"),
    transaction_count=("TransactionID", "count")
).reset_index()

In [ ]:
# Normalize the data
scaler = StandardScaler()
numerical_features = ["total_spent", "avg_quantity", "transaction_count"]
customer_summary_scaled = scaler.fit_transform(customer_summary[numerical_features])

In [ ]:
# Calculate similarity scores
similarity_matrix = cosine_similarity(customer_summary_scaled)
similarity_df = pd.DataFrame(similarity_matrix, index=customer_summary["CustomerID"], columns=customer_summary["CustomerID"])


In [ ]:
# Generate top 3 lookalikes for the first 20 customers
lookalike_results = {}
for customer in customer_summary["CustomerID"].head(20):
    similar_customers = similarity_df[customer].sort_values(ascending=False)[1:4]
    lookalike_results[customer] = list(similar_customers.items())

In [ ]:
# Save results to Lookalike.csv
lookalike_df = pd.DataFrame([
    {"CustomerID": cust, "Lookalikes": lookalikes}
    for cust, lookalikes in lookalike_results.items()
])
lookalike_df.to_csv("FirstName_LastName_Lookalike.csv", index=False)

print("Lookalike Model Results Saved to CSV.")